In [7]:
import os
import numpy as np
import pandas as pd

#from subprocess import check_output
#print(check_output(['ls', '/iceberg/processed']).decode('utf8'))

## Credits to the following awesome authors and kernels

Author: QuantScientist

File: sub_200_ens_densenet.csv

Link: https://www.kaggle.com/solomonk/pytorch-cnn-densenet-ensemble-lb-0-1538

Author: wvadim

File: sub_TF_keras.csv

Link: https://www.kaggle.com/wvadim/keras-tf-lb-0-18

Author: Ed Miller

File: sub_fcn.csv

Link: https://www.kaggle.com/bluevalhalla/fully-convolutional-network-lb-0-193

Author: Chia-Ta Tsai

File: sub_blend009.csv

Link: https://www.kaggle.com/cttsai/ensembling-gbms-lb-203

Author: DeveshMaheshwari

File: sub_keras_beginner.csv

Link: https://www.kaggle.com/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d

Author: Submarineering

Files: submission38.csv , submission43.csv

Link : https://www.kaggle.com/submarineering/submission38-lb01448

Without their truly dedicated efforts, this notebook will not be possible.

In [10]:
sub_path = 'iceberg/processed'
all_files =os.listdir(sub_path)
all_files = all_files[1:3]
all_files.append('submission38.csv')
all_files.append('submission42.csv')
all_files

['test.json', 'train.json', 'submission38.csv', 'submission42.csv']

In [13]:
# Read and concatenate submissions
out1 = pd.read_csv("iceberg/processed/sub_200_ens_densenet.csv", index_col=0)
out2 = pd.read_csv("iceberg/processed/sub_TF_keras.csv", index_col=0)
out3 = pd.read_csv("iceberg/processed/submission38.csv", index_col=0)
out4 = pd.read_csv("iceberg/processed/submission43.csv", index_col=0)
concat_sub = pd.concat([out1, out2, out3, out4], axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3
0,5941774d,1.943216e-02,0.005586,0.008875,0.005586
1,4023181e,3.168809e-02,0.145927,0.146779,0.031688
2,b20200e4,4.000000e-08,0.000015,0.048646,0.001000
3,e7f018bb,9.925741e-01,0.999914,0.999184,0.999000
4,4371c8c3,2.215107e-02,0.033843,0.277251,0.022151


In [14]:
concat_sub.corr()

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3
is_iceberg_0,1.000000,0.890434,0.986291,0.994880
is_iceberg_1,0.890434,1.000000,0.923848,0.900871
is_iceberg_2,0.986291,0.923848,1.000000,0.988002
is_iceberg_3,0.994880,0.900871,0.988002,1.000000


In [15]:
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:6].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:6].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:6].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:6].median(axis=1)

In [16]:
cutoff_lo = 0.8
cutoff_hi = 0.2

## Mean Stacking

In [17]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_mean']
concat_sub[['id', 'is_iceberg']].to_csv('stack_mean.csv', index=False, float_format='%.6f')

## Median Stacking

In [18]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_median']
concat_sub[['id', 'is_iceberg']].to_csv('stack_median.csv', index=False, float_format='%.6f')

## PushOut + Median Stacking

In [19]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 1, 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             0, concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_pushout_median.csv', 
                                        index=False, float_format='%.6f')

## MinMax + Mean Stacking

In [20]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_mean']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_mean.csv', 
                                        index=False, float_format='%.6f')

## MinMax + Median Stacking



In [21]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub['is_iceberg'] = np.clip(concat_sub['is_iceberg'].values, 0.001, 0.999)
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_median.csv', 
                                       index=False, float_format='%.6f')

## MinMax + BestBase Stacking

In [22]:
sub_base = pd.read_csv('iceberg/processed/submission43.csv')

In [23]:
concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:4]>cutoff_lo, axis=1),
                                   concat_sub['is_iceberg_max'],np.where(np.all(concat_sub.iloc[:,1:4]<cutoff_hi,axis=1),concat_sub['is_iceberg_min'],concat_sub['is_iceberg_base']))
concat_sub['is_iceberg'] = np.clip(concat_sub['is_iceberg'].values,0.001,0.999)
concat_sub[['id','is_iceberg']].to_csv('submission54.csv',index=False,float_format='%.6f')